In [1]:
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import os, shutil, glob, os.path
from collections import defaultdict 
import torch

In [2]:
from model import EncoderCNN, DecoderRNN

In [9]:
os.chdir('E:/insta filter/')

from data_loader import get_loader
import matplotlib
matplotlib.use('tkagg')
transform_test = transforms.Compose([ 
    transforms.Resize(256),                          
    transforms.RandomCrop(224),                      
    transforms.RandomHorizontalFlip(),               
    transforms.ToTensor(),                           
    transforms.Normalize((0.485, 0.456, 0.406),      
                         (0.229, 0.224, 0.225))])

# Create the data loader.
data_loader = get_loader(transform=transform_test, mode='test',ipath = '11.jpg')
orig_image, image = next(iter(data_loader))
plt.imshow(np.squeeze(orig_image))
plt.title('example image')
plt.show()


In [10]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder_file = 'encoder-3.pkl'
decoder_file = 'decoder-3.pkl'

embed_size = 256
hidden_size = 512

vocab_size = len(data_loader.dataset.vocab)

encoder = EncoderCNN(embed_size)
encoder.eval()
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
decoder.eval()

encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file),map_location=torch.device('cpu')))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file),map_location=torch.device('cpu')))

encoder.to(device)
decoder.to(device)

image = image.to(device)

features = encoder(image).unsqueeze(1)

output = decoder.sample(features)
print('example output:', output)

assert (type(output)==list), "Output needs to be a Python list" 
assert all([type(x)==int for x in output]), "Output should be a list of integers." 
assert all([x in data_loader.dataset.vocab.idx2word for x in output]), "Each entry in the output needs to correspond to an integer that indicates a token in the vocabulary."

def clean_sentence(output):
    sentence = ''
    new_output = output[1:-1] 
    for i in range(len(new_output)):
        curr_token = new_output[i]
        if curr_token == 18: sentence += data_loader.dataset.vocab.idx2word[curr_token]
        else: sentence += ' ' + data_loader.dataset.vocab.idx2word[curr_token]
    return sentence

sentence = clean_sentence(output)
print('example sentence:', sentence)

assert type(sentence)==str, 'Sentence needs to be a Python string!'

def get_prediction():
    orig_image, image = next(iter(data_loader))
    plt.imshow(np.squeeze(orig_image))
    plt.title('Sample Image')
    plt.show()
    image = image.to(device)
    features = encoder(image).unsqueeze(1)
    output = decoder.sample(features)    
    sentence = clean_sentence(output)
    print(sentence)

get_prediction()

example output: [0, 3, 821, 13, 109, 21, 3, 3124, 6, 3, 185, 13, 4829, 18, 1]
example sentence:  a plate of food with a sandwich and a side of fries.
 a plate of food with a sandwich and a side of fries.
